<a href="https://colab.research.google.com/github/jelal1cam/CAMSIF/blob/main/S%26P_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install fredapi itables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.0 MB/s eta 0:00:00


In [8]:
import yfinance as yf
from prophet import Prophet
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error
from fredapi import Fred
from itables import show
import scipy.stats as stats
from itables import options as opt

# Define your FRED API key here
fred = Fred(api_key="6ee76636ad49b704cff4c22a8416dae2")

def fetch_data(symbol, start, end):
    data = yf.download(symbol, start=start, end=end)
    df = data.reset_index()[['Date', 'Close']]
    df.columns = ['ds', 'y']
    return df

end_date = '2023-09-20'

# Fetching FRED data
gdp_growth = fred.get_series('A191RL1Q225SBEA', start='2000-01-01', end=end_date).reset_index()  # Quarterly
gdp_growth.columns = ['ds', 'gdp_growth']
unemployment_rate = fred.get_series('UNRATE', start='2000-01-01', end=end_date).reset_index()  # Monthly
unemployment_rate.columns = ['ds', 'unemployment_rate']
dff = fred.get_series('DFF', start='2000-01-01', end=end_date).reset_index()
dff.columns = ['ds', 'DFF']

# Fetching yahoo finance data
snp500 = fetch_data('^GSPC', '2000-01-01', end_date)
snp500.columns = ['ds', 'closing_price']
vix = fetch_data('^VIX', '2000-01-01', end_date)
vix.columns = ['ds', 'VIX']
us_dollar_index = fetch_data('DX-Y.NYB', '2000-01-01', end_date)
us_dollar_index.columns = ['ds', 'us_dollar_index']

# Merging the data
merged_data = snp500.merge(gdp_growth, on='ds', how='left').merge(unemployment_rate, on='ds', how='left').merge(vix, on= 'ds', how='left').merge(dff, on='ds', how='left').merge(us_dollar_index, on='ds', how='left')

# Forward filling NaN values
merged_data['unemployment_rate'] = merged_data['unemployment_rate'].ffill()
merged_data['gdp_growth'] = merged_data['gdp_growth'].ffill()

# Dropping NaN values
merged_data.dropna(inplace=True)

opt.lengthMenu = [5, 10, 20]
opt.maxBytes = 0
opt.showIndex = True

show(merged_data)

# Interactive Plotting using plotly
fig = go.Figure()

# Add traces for each column
for col in merged_data.columns:
    if col != 'ds':
        fig.add_trace(go.Scatter(x=merged_data['ds'], y=merged_data[col], mode='lines', name=col))

fig.update_layout(title='Time Series of S&P 500 Closing Price and Regressors', xaxis_title='Date', yaxis_title='Value')
fig.show()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
# Forecasting function for Prophet
def forecast_prophet(data, periods, regressors=None):
    model = Prophet(daily_seasonality=True)

    # Rename columns if it's not in the format 'ds' and 'y'
    if 'y' not in data.columns:
        data = data.rename(columns={data.columns[1]: 'y'})

    # Add additional regressors if any
    if regressors:
        for reg in regressors:
            model.add_regressor(reg)

    model.fit(data)
    future = model.make_future_dataframe(periods=periods)

    # If there are regressors, we predict them for future and merge to the main future dataframe
    if regressors:
        for reg in regressors:
            reg_forecast = forecast_prophet(data[['ds', reg]], periods)
            future = future.merge(reg_forecast[['ds', 'yhat']], on='ds', how='left').rename(columns={'yhat': reg})

    forecast = model.predict(future)
    return forecast

# Forecast each regressor
gdp_forecast = forecast_prophet(merged_data[['ds', 'gdp_growth']], 548)
unemployment_forecast = forecast_prophet(merged_data[['ds', 'unemployment_rate']], 548)
vix_forecast = forecast_prophet(merged_data[['ds', 'VIX']], 548)
dff_forecast = forecast_prophet(merged_data[['ds', 'DFF']], 548)
us_dollar_index_forecast = forecast_prophet(merged_data[['ds', 'us_dollar_index']], 548)

# Append the regressor forecasts to the original dataframe
merged_data_forecast = merged_data.copy()
for forecast_df, col in zip([gdp_forecast, unemployment_forecast, vix_forecast, dff_forecast, us_dollar_index_forecast], ['gdp_growth', 'unemployment_rate', 'VIX', 'DFF', 'us_dollar_index']):
    merged_data_forecast = merged_data_forecast.merge(forecast_df[['ds', 'yhat']], on='ds', how='left').rename(columns={'yhat': f'{col}_forecast'})

# Use the forecasts as regressors to predict the S&P 500 closing price
closing_price_forecast = forecast_prophet(merged_data_forecast, 548, regressors=['gdp_growth_forecast', 'unemployment_rate_forecast', 'VIX_forecast', 'DFF_forecast', 'us_dollar_index_forecast'])

# Plotting using fan chart logic
fig = go.Figure()

# Use 'Close' from merged_data for plotting the actual data
fig.add_trace(go.Scatter(x=merged_data['ds'], y=merged_data['closing_price'], name="Actual", line=dict(color='black')))
fig.add_trace(go.Scatter(x=closing_price_forecast['ds'], y=closing_price_forecast['yhat'], name="Median Forecast", line=dict(color='red')))

# Fan chart based on Prophet's confidence intervals
fig.add_trace(go.Scatter(
    x=closing_price_forecast['ds'],
    y=closing_price_forecast['yhat_upper'],
    fill=None,
    mode='lines',
    line=dict(color='rgba(0, 100, 200, 0.2)'),
    name='Upper Bound'
))

fig.add_trace(go.Scatter(
    x=closing_price_forecast['ds'],
    y=closing_price_forecast['yhat_lower'],
    fill='tonexty',
    mode='lines',
    line=dict(color='rgba(0, 100, 200, 0.2)'),
    name='Lower Bound'
))

fig.update_layout(title="S&P 500 Forecast with Fan Chart", xaxis_title="Date", yaxis_title="S&P 500 Close Price")
fig.show()

DEBUG:cmdstanpy:input tempfile: /tmp/tmp8fb685il/moneajgl.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8fb685il/543j_f8i.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=84486', 'data', 'file=/tmp/tmp8fb685il/moneajgl.json', 'init=/tmp/tmp8fb685il/543j_f8i.json', 'output', 'file=/tmp/tmp8fb685il/prophet_model95by5b_1/prophet_model-20230922223710.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
22:37:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:37:11 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8fb685il/lgi0_2l0.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8fb685il/0du801x9.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [10]:
# Choose a specific date
target_date = '2024-12-31'
forecast_row = closing_price_forecast[closing_price_forecast['ds'] == target_date].iloc[0]

# Extract the median, lower, and upper prediction
median = forecast_row['yhat']
lower = forecast_row['yhat_lower']
upper = forecast_row['yhat_upper']

# Parameters for log-normal: mu and sigma are mean and std deviation of the underlying normal distribution
mu = np.log(median**2 / np.sqrt(upper*lower))
sigma = np.sqrt(np.log(upper/lower))

# Calculate the mode (most likely value) of the log-normal distribution
mode = np.exp(mu - sigma**2)

# Plotting
x = np.linspace(0, upper + (upper - lower), 1000)
pdf = stats.lognorm.pdf(x, s=sigma, scale=np.exp(mu))

fig = go.Figure()

# Probability density curve
fig.add_trace(go.Scatter(x=x, y=pdf, mode='lines', name='Probability Density'))

# Mode (Most likely closing price)
fig.add_trace(go.Scatter(x=[mode], y=[stats.lognorm.pdf(mode, s=sigma, scale=np.exp(mu))],
                         mode='markers', marker=dict(size=10, color='red', symbol='cross'),
                         name='Most Likely Price'))

# Add confidence intervals
confidence_levels = [0.99, 0.95, 0.68]
colors = ['rgba(0, 128, 0, 0.2)', 'rgba(0, 128, 0, 0.3)', 'rgba(0, 128, 0, 0.4)']

for level, color in zip(confidence_levels, colors):
    lower_bound = stats.lognorm.ppf((1 - level) / 2, s=sigma, scale=np.exp(mu))
    upper_bound = stats.lognorm.ppf(1 - (1 - level) / 2, s=sigma, scale=np.exp(mu))

    mask = (x >= lower_bound) & (x <= upper_bound)

    fig.add_trace(go.Scatter(
        x=x[mask],
        y=pdf[mask],
        fill='tozeroy',
        fillcolor=color,
        mode='none',
        name=f"{int(level*100)}% Confidence Interval",
        showlegend=False
    ))

fig.update_layout(title=f"Predicted Log-normal Distribution for {target_date}",
                  xaxis_title="S&P 500 Close Price",
                  yaxis_title="Probability Density",
                  hovermode="x")

fig.show()